In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import splink

In [ ]:
from splink.duckdb.duckdb_linker import DuckDBLinker
from try_settings import settings_dict
import pandas as pd


from try_settings import settings_dict


df = pd.read_csv("./tests/datasets/fake_1000_from_splink_demos.csv")


linker = DuckDBLinker(settings_dict, input_tables={"input_df": df})

linker.train_u_using_random_sampling(target_rows=1e6)

blocking_rule = "l.first_name = r.first_name and l.surname = r.surname"
linker.train_m_using_expectation_maximisation(blocking_rule)

blocking_rule = "l.dob = r.dob"
linker.train_m_using_expectation_maximisation(blocking_rule)


linker.compute_tf_table("first_name")
linker.compute_tf_table("city")





In [ ]:
from splink.charts import waterfall_chart

import ipywidgets as widgets

@widgets.interact(first_name='Julia', surname="Taylor", dob="2015-10-29", city="London", email="hannah88@powers.com")
def interactive_link(first_name, surname, dob, city, email):    

    record = {'unique_id': 123987,
     'first_name': first_name,
     'surname': surname,
     'dob': dob,
     'city': city,
     'email': email,
     'group': 0}

    for key in record.keys():
        if type(record[key]) == str:
            if record[key].strip() == "":
                record[key] = None

    
    df_inc = linker.find_matches_to_new_records([record], blocking_rules=[f"(true)"]).as_pandas_dataframe()
    df_inc = df_inc.sort_values("match_weight", ascending=False)
    recs = df_inc.to_dict(orient="records")
    


    waterfall_chart(recs, linker.settings_obj, filter_nulls=False)


In [ ]:
record = {'unique_id': 123987,
 'first_name': "Julia ",
 'surname': "Taylor",
 'dob': "2015-10-29",
 'city': "London",
 'email': "hannah88@powers.com",
 'group': 0}


pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

df_inc = linker.find_matches_to_new_records([record], blocking_rules=[f"(true )"]).as_pandas_dataframe()
df_inc